In [1]:
import pandas as pd
import numpy as np

In [2]:
file_school= "Resources/schools_complete.csv"
file_student= "Resources/students_complete.csv"

schools_df = pd.read_csv(file_school)
students_df = pd.read_csv(file_student)

In [3]:
schools_df.head()

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [4]:
students_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [5]:
# merge school and student data
school_student_df = pd.merge(schools_df, students_df, on = ['school_name','school_name'], how='right')
school_student_df.head()

,School ID,school_name,type,size,budget,Student ID,student_name,gender,grade,reading_score,math_score
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84


In [6]:
#total number of schools
total_schools = school_student_df['school_name'].nunique()

In [7]:
#total number of students
total_students = students_df['student_name'].count()

In [8]:
#total budget amount for district
total_budget = schools_df['budget'].sum()

In [9]:
#average district math score calculation
average_math_scores = students_df["math_score"].sum()/students_df["math_score"].count()

In [10]:
#average district reading score calculation
average_reading_scores = students_df["reading_score"].sum()/students_df["reading_score"].count()

In [11]:
# datatframe of students who scored 70 or higher on math test
pass__math_df = students_df.loc[students_df["math_score"]>=70]
#percentage of students with passing math score (70 or higher)
percent_passing_math = pass__math_df["math_score"].count()/students_df["math_score"].count()*100


In [12]:
# datatframe of students who scored 70 or higher on reading test
pass__reading_df = students_df.loc[students_df["reading_score"]>=70]
#percentage of students with passing reading score (70 or higher)
percent_passing_reading = pass__reading_df["reading_score"].count()/students_df["reading_score"].count()*100

In [13]:
# datatframe of students who scored 70 or higher on both math and reading test
students_pass__both_df = students_df.loc[(students_df["math_score"]>=70) & (students_df["reading_score"]>=70)]
#percentage of students with passing score on both math and reading test
percent_passing_both = students_pass__both_df["reading_score"].count()/students_df["reading_score"].count()*100

In [14]:
#creates district summary table
summary = [{"Total Schools": total_schools,"Total Students": total_students, "Total Budget": total_budget, "Average Math Score": average_math_scores, "Average Reading Score":average_reading_scores,  "% Passing Math": percent_passing_math, "% Passing Reading": percent_passing_reading, " % Overall Passing": percent_passing_both }]
district_summary_df = pd.DataFrame(summary)
#format summary table
district_summary_df['Total Students'] = district_summary_df['Total Students'].map("{:,}".format)
district_summary_df['Total Budget'] = district_summary_df['Total Budget'].map("${:0,.2f}".format)
district_summary_df = np.round(district_summary_df, decimals=2)
#final District Summary table
district_summary_df.head()

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98,85.81,65.17


In [15]:
#create data frame with school, type and mean for each category
grouped_school_data = school_student_df.groupby(['school_name','type'])
grouped_school_data= grouped_school_data.mean()
grouped_school_data_df = pd.DataFrame(grouped_school_data)
#calculate per student budget, add to database
grouped_school_data_df['Per Student Budget'] = grouped_school_data_df['budget']/grouped_school_data_df['size']

In [16]:
#create column using bins to determine if student passed math/reading test
score_bins = [0,69,100]
score_labels = ['No', 'Yes']
school_student_df["passing_math"]= pd.cut(school_student_df["math_score"], score_bins, labels = score_labels)
school_student_df["passing_reading"]= pd.cut(school_student_df["reading_score"], score_bins, labels = score_labels)
new_data_df = pd.DataFrame(school_student_df)

In [17]:
#school's math passing rate calculation
passing_math_scores = school_student_df[(school_student_df['passing_math']=='Yes')]
passing_math_scores_per_school= passing_math_scores.groupby('school_name')
total_passing_math_scores = passing_math_scores_per_school['math_score'].count()
school_math_pass_rate = total_passing_math_scores/grouped_school_data['size']*100

In [18]:
#school's reading passing rate calculation
passing_read_scores = school_student_df[(school_student_df['passing_reading']=='Yes')]
passing_read_scores_per_school= passing_read_scores.groupby('school_name')
total_passing_read_scores = passing_read_scores_per_school['reading_score'].count()
school_read_pass_rate = total_passing_read_scores/grouped_school_data['size']*100

In [19]:
#school's overall passing rate calculation
passing_both_scores = school_student_df[(school_student_df['passing_math']=='Yes')&(school_student_df['passing_reading']=='Yes')]
passing_both_scores_per_school= passing_both_scores.groupby('school_name')
total_passing_both_scores = passing_both_scores_per_school['reading_score'].count()
school_overall_pass_rate = total_passing_both_scores/grouped_school_data['size']*100

In [20]:
#create summary table by adding passing rates to database grouped by school and type
grouped_school_data_df['%Passing Math'] = school_math_pass_rate
grouped_school_data_df['% Passing Reading'] = school_read_pass_rate
grouped_school_data_df['% Overall Passing' ] = school_overall_pass_rate

In [21]:
#format summary table
school_summary = grouped_school_data_df.rename(columns = {'size': 'Total Students','budget': 'Total School Budget','math_score': 'Average Math Score', 'reading_score': 'Average Reading Score'})
#reorder columns
school_summary = school_summary[['Total Students','Total School Budget','Per Student Budget', 'Average Math Score','Average Reading Score',"%Passing Math", "% Passing Reading","% Overall Passing"]]
school_summary_df = pd.DataFrame(school_summary)
school_summary_df = school_summary_df.rename_axis(index=['School Name', 'School Type']) #rename indices 
school_summary_df = np.round(school_summary_df, decimals = 2)
#more formatting of summary table, change objects to float
school_summary_df['Per Student Budget'] = school_summary_df['Per Student Budget'].map("${:0,.2f}".format)
school_summary_df['Total Students'] = school_summary_df['Total Students'].apply(np.int64) 
school_summary_df['Total School Budget'] = pd.to_numeric(school_summary_df['Total School Budget'],errors='coerce') #change object to float to format
#final School Summary overview table
school_summary_df['Total School Budget'] = school_summary_df['Total School Budget'].map("${:0,.2f}".format)
school_summary_df.head()

,,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,%Passing Math,% Passing Reading,% Overall Passing
School Name,School Type,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.05,81.03,66.68,81.93,54.64
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13,97.04,91.33
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,65.99,80.74,53.20
Ford High School,District,2739,"$1,763,916.00",$644.00,77.10,80.75,68.31,79.30,54.29
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39,97.14,90.60


In [22]:
#sort and display the Top Five Performing Schools by %overall passing
top_schools_summary = school_summary_df.sort_values('% Overall Passing', ascending = False)
top_schools_summary.head()

,,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,%Passing Math,% Passing Reading,% Overall Passing
School Name,School Type,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13,97.04,91.33
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.42,83.85,93.27,97.31,90.95
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39,97.14,90.60
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.27,83.99,93.87,96.54,90.58
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59,95.95,90.54


In [23]:
#sort and display the Bottom Performing Schools by %overall passing
bottom_performing_schools =school_summary_df.sort_values('% Overall Passing')
bottom_performing_schools.head()

,,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,%Passing Math,% Passing Reading,% Overall Passing
School Name,School Type,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.84,80.74,66.37,80.22,52.99
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,65.99,80.74,53.20
Huang High School,District,2917,"$1,910,635.00",$655.00,76.63,81.18,65.68,81.32,53.51
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.29,80.93,66.75,80.86,53.53
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.07,80.97,66.06,81.22,53.54


In [24]:
#create series for math scores of grade 9
math_scores_grade9 = school_student_df[(school_student_df['grade']=='9th')]
math_scores_grade9 = math_scores_grade9.groupby('school_name')
math_scores_grade9 = math_scores_grade9['math_score'].mean()


In [25]:
#create series for math scores of grade 10
math_scores_grade10 = school_student_df[(school_student_df['grade']=='10th')]
math_scores_grade10 = math_scores_grade10.groupby('school_name')
math_scores_grade10 = math_scores_grade10['math_score'].mean()


In [26]:
#create series for math scores of grade 11
math_scores_grade11 = school_student_df[(school_student_df['grade']=='11th')]
math_scores_grade11 = math_scores_grade11.groupby('school_name')
math_scores_grade11 = math_scores_grade11['math_score'].mean()


In [27]:
#create series for math scores of grade 12
math_scores_grade12 = school_student_df[(school_student_df['grade']=='12th')]
math_scores_grade12 = math_scores_grade12.groupby('school_name')
math_scores_grade12 = math_scores_grade12['math_score'].mean()


In [28]:
#Math Scores by Grade data frame
math_scores_by_grade = pd.concat([math_scores_grade9, math_scores_grade10, math_scores_grade11, math_scores_grade12], axis=1)
math_scores_by_grade_df = pd.DataFrame(math_scores_by_grade)
math_scores_by_grade_df = math_scores_by_grade_df.rename_axis(index=['School Name']) #rename index
math_scores_by_grade_df.columns = ['9th', '10th', '11th', '12th'] 
math_scores_by_grade_df = np.round(math_scores_by_grade_df , decimals = 2)
math_scores_by_grade_df.head()

,9th,10th,11th,12th
School Name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36


In [29]:
#create series for reading scores of grade 9
read_scores_grade9 = school_student_df[(school_student_df['grade']=='9th')]
read_scores_grade9 = read_scores_grade9.groupby('school_name')
read_scores_grade9 = read_scores_grade9['reading_score'].mean()

In [30]:
#create series for reading scores of grade 10
read_scores_grade10 = school_student_df[(school_student_df['grade']=='10th')]
read_scores_grade10 = read_scores_grade10.groupby('school_name')
read_scores_grade10 = read_scores_grade10['reading_score'].mean()

In [31]:
#create series for reading scores of grade 11
read_scores_grade11 = school_student_df[(school_student_df['grade']=='11th')]
read_scores_grade11 = read_scores_grade11.groupby('school_name')
read_scores_grade11= read_scores_grade11['reading_score'].mean()

In [32]:
#create series for reading scores of grade 12
read_scores_grade12 = school_student_df[(school_student_df['grade']=='12th')]
read_scores_grade12 = read_scores_grade12.groupby('school_name')
read_scores_grade12= read_scores_grade12['reading_score'].mean()

In [33]:
#Reading Scores by Grade data frame
read_scores_by_grade = pd.concat([read_scores_grade9, read_scores_grade10, read_scores_grade11,read_scores_grade12], axis=1)
read_scores_by_grade_df = pd.DataFrame(read_scores_by_grade)
read_scores_by_grade_df = read_scores_by_grade_df.rename_axis(index=['School Name']) #rename index
read_scores_by_grade_df.columns = ['9th', '10th', '11th', '12th'] 
read_scores_by_grade_df = np.round(read_scores_by_grade_df , decimals = 2)
read_scores_by_grade_df.head()

,9th,10th,11th,12th
School Name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01


In [34]:
#create bins based on spending ranges per student
spending_bins = [0,584,629, 644, 675]
spending_labels = ["< $584", "$585 - $629", "$630 - $644", "$645 - $675"]
grouped_school_data_df["Spending Ranges (Per Student)"]= pd.cut(grouped_school_data_df["Per Student Budget"], spending_bins, labels = spending_labels)
spending_df = pd.DataFrame(grouped_school_data_df)

In [35]:
#create table that breaks down school performance based on average spending ranges
school_spending = spending_df.groupby('Spending Ranges (Per Student)')
school_spending =school_spending.mean()
scores_by_school_spending_df= pd.DataFrame(school_spending)
#drop columns from scores by school spending data frame
school_spending_summary = scores_by_school_spending_df.drop(columns=['School ID','size','budget','Student ID', 'Per Student Budget'])
school_spending_summary_df = pd.DataFrame(school_spending_summary)
school_spending_summary_df = np.round(school_spending_summary_df, decimals = 2)
#format Scores by School Spending table
school_spending_summary_df = school_spending_summary_df.rename(columns = {'math_score': 'Average Math Score', 'reading_score': 'Average Reading Score'})  
school_spending_summary_df.head()

,Average Reading Score,Average Math Score,%Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
< $584,83.93,83.46,93.46,96.61,90.37
$585 - $629,83.16,81.90,87.13,92.72,81.42
$630 - $644,81.62,78.52,73.48,84.39,62.86
$645 - $675,81.03,77.00,66.16,81.13,53.53


In [36]:
#create bins based on school size
size_bins = [0,1000,2000, 5000]
size_labels = ["Small(< 1000)", "Medium(1000-2000)", "Large(2000 -5000)"]
grouped_school_data_df["School Size"]= pd.cut(grouped_school_data_df["size"], size_bins, labels = size_labels)
school_size_df= pd.DataFrame(grouped_school_data_df)

In [37]:
#group schools by size and create data frame
size_scores = school_size_df.groupby('School Size')
school_size_scores = size_scores.mean()
school_size_scores_df = pd.DataFrame(school_size_scores)

In [38]:
#format Scores by School Size table
school_size_scores_df = school_size_scores_df.drop(columns=['School ID','size','budget','Student ID', 'Per Student Budget'])
school_size_scores_df= school_size_scores_df.rename(columns = {'math_score': 'Average Math Score', 'reading_score': 'Average Reading Score'})  
school_size_scores_df = np.round(school_size_scores_df, decimals = 2)
school_size_scores_df.head()

,Average Reading Score,Average Math Score,%Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small(< 1000),83.93,83.82,93.55,96.10,89.88
Medium(1000-2000),83.86,83.37,93.60,96.79,90.62
Large(2000 -5000),81.34,77.75,69.96,82.77,58.29


In [39]:
#group schools by type and create data frame
school_type= grouped_school_data_df.groupby('type')
school_type_table = school_type.mean()
scores_by_school_type_df = pd.DataFrame(school_type_table)       
scores_by_school_type_df = scores_by_school_type_df.rename_axis(index=['School Type'])

In [40]:
#format Scores by School Type table
scores_by_school_type_df= scores_by_school_type_df.drop(columns=['School ID','size','budget','Student ID', 'Per Student Budget'])
scores_by_school_type_df = scores_by_school_type_df.rename(columns = {'math_score': 'Average Math Score', 'reading_score': 'Average Reading Score'})  
scores_by_school_type_df = np.round(scores_by_school_type_df, decimals = 2)
scores_by_school_type_df.head()

,Average Reading Score,Average Math Score,%Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.90,83.47,93.62,96.59,90.43
District,80.97,76.96,66.55,80.80,53.67
